In [1]:
# https://haptik.ai/tech/offline-on-device-ml-text-classification/

In [2]:
import csv
import re
import json
import numpy as np

In [3]:
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from keras.layers import Dense, Input, Dropout
from tensorflow.python.keras import models, optimizers, losses, activations
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.models import Model

Using TensorFlow backend.


In [4]:
sentences, labels = [], []
with open("chatbot.csv", "r") as f:
    data = csv.reader(f)
    for row in data:
        sentences.append(row[0])
        labels.append(row[1])

In [5]:
sentences = [re.sub(r".,:?{}", " ", sentence) for sentence in sentences]

corpus = " ".join(sentences)
words = set(corpus.split())
word_index = {word: index for index, word in enumerate(words)}
with open("word_index.json", "w") as file:
    json.dump(word_index, file)

In [6]:
LE = LabelEncoder()

In [7]:
def train_and_eval(sentences, labels):
    labels = LE.fit_transform(labels)
    labels = np.array(labels)
    num_classes = len(labels)
    onehot_labels = tf.keras.utils.to_categorical(labels, num_classes=num_classes)

    setences_tokens = [sentence.split() for sentence in sentences]
    tokenizer = tf.keras.preprocessing.text.Tokenizer()
    tokenizer.word_index = word_index
    sentences_features = tokenizer.texts_to_matrix(setences_tokens)
    train_features, val_features, train_labels, val_labels = train_test_split(
        sentences_features, onehot_labels, test_size=0.1
    )
    feature_input = Input(shape=(sentences_features.shape[1],))
    dense = Dense(128, activation=activations.relu)(feature_input)
    merged = BatchNormalization()(dense)
    merged = Dropout(0.2)(merged)
    merged = Dense(64, activation=activations.relu)(merged)
    merged = BatchNormalization()(merged)
    merged = Dropout(0.2)(merged)
    preds = Dense(num_classes, activation=activations.softmax)(merged)
    model = Model(inputs=[word_index], outputs=preds)

    model.compile(
        loss=losses.categorical_crossentropy, optimizer="nadam", metrics=["acc"]
    )

    early_stopping = EarlyStopping(monitor="val_loss", patience=5)
    model.fit(
        [train_features],
        train_labels,
        validation_data=([val_features], val_labels),
        epochs=200,
        batch_size=8,
        shuffle=True,
        callbacks=[early_stopping],
    )
    model.save("models.h5")

In [8]:
train_and_eval(sentences, labels)

ValueError: Input tensors to a Model must come from `keras.layers.Input`. Received: {'munich': 0, 'subway': 1, 'kieferngarte': 2, 'is': 3, "what's": 4, 'marienplatz?': 5, 'laim': 6, 'sued': 7, 'to': 8, 'station': 9, 'prinzregentenplatz': 10, 'munchner': 11, 'in': 12, 'winterstrasse': 13, 'poccistraße': 14, 'heide': 15, '?': 16, 'platz': 17, 'ostbahnhof': 18, 'departs': 19, 'how': 20, 'can': 21, 'me': 22, 'spitzingplatz': 23, 'i': 24, 'einkaufszentrum': 25, 'please.': 26, 'theresienstrasse': 27, 'airport': 28, 'winterstraße': 29, 'muenchen': 30, 'strassäcker': 31, 'odeonsplatz': 32, 'freiheit?': 33, 'klinikum': 34, 'central': 35, 'from': 36, 'poccistraße?': 37, 'find': 38, 'freimann?': 39, 'you': 40, 'rotkreuzplatz': 41, 'leave': 42, 'olympia': 43, 'hauptbahnhof?': 44, '21': 45, 'alte': 46, 'city': 47, 'when': 48, 'münchner': 49, 'untere': 50, 'neuperlach': 51, 'muncher': 52, 'bonner': 53, 'freicheit': 54, 'u6': 55, 'depart': 56, 'foschungszentrum': 57, 'take': 58, 'end:garching': 59, '

In [ ]:
def test(sentence, model_path, word_index_path):

    classifier = models.load_model( 'models/models.h5' )
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='.,:?{} ')
    sentences = re.sub(r'.,:?{}', ' ', sentence)
    with open(word_index_path, 'r') as f:
        tokenizer.word_index = json.loads(f.read())
        tokenized_messages = tokenizer.texts_to_matrix(sentence.split())
        p = list(classifier.predict(tokenized_messages)[0])

    for index, each in enumerate(p):
        print(index, each)

In [ ]:
def convert_model_to_tflite(keras_model_path):

    tf.logging.set_verbosity( tf.logging.ERROR )
    converter = tf.contrib.lite.TFLiteConverter.from_keras_model_file(keras_model_path )

    converter.post_training_quantize = True
    tflite_buffer = converter.convert()
    open( 'model.tflite' , 'wb' ).write( tflite_buffer )

    print( 'TFLite model created.')